# Maps coversion to TOPOJSON

## This notebook aims to:
    - [X] Converter cada shapefile de mapa para seu formato topojson
    - [X] Produzir um único JSON que contém a relação dos topojson e ano

## Notes:
    - Esse notebook usa os arquivos criados em ./2017-01-10-bf-historical-maps.ipynb
    - Esse notebook tem como dependências os binários 'ogr2ogr', 'geo2topo' e 'toposimplify'
    - O formato do arquivo final é um JSON na estrutura:
        [{"year": < ano >}]

## Converta mapas para GeoJSON

In [1]:
%%bash
infolder=../data/historical_maps
outfolder=../data/maps_conversion
mkdir $outfolder
for map_year in $(ls $infolder); do
    mkdir $outfolder/$map_year
    ogr2ogr -f GeoJSON -t_srs "EPSG:4326" $outfolder/$map_year/states.json $infolder/$map_year/*.shp
done

Lembrando da observação feita no notebook anterior, eu baixei o mapa de 1950 para representar 1956, já que as fronteiras são as mesmas, mas para ficar correto, tenho que corrigir o nome de Território de Guaporé para Território de Rondônia

In [2]:
%%bash
sed -i -e 's/Território do Guaporé/Território de Rondônia/' ../data/maps_conversion/1956/states.json

## Converta para TopoJSON

In [3]:
%%bash
folder=../data/maps_conversion
for map_year in $(ls $folder); do
    geo2topo -o $folder/$map_year/states.topo.json -- $folder/$map_year/states.json
done

## Simplifique os arquivos

In [4]:
%%bash
folder=../data/maps_conversion
for map_year in $(ls $folder); do
    toposimplify $folder/$map_year/states.topo.json -P 0.1 -f \
        -o $folder/$map_year/states_simplified.topo.json
done

equivalent --planar-area 0.004864058560179687
equivalent --planar-area 0.0046958326918853585
equivalent --planar-area 0.0046958326918853585
equivalent --planar-area 0.004840564706383291
equivalent --planar-area 0.00446150815736295
equivalent --planar-area 0.0043850919071649124
equivalent --planar-area 0.004355568956756782


In [5]:
%%bash
du -h ../data/maps_conversion/*/*.json

1,5M	../data/maps_conversion/1940/states.json
152K	../data/maps_conversion/1940/states_simplified.topo.json
1,3M	../data/maps_conversion/1940/states.topo.json
1,7M	../data/maps_conversion/1950/states.json
172K	../data/maps_conversion/1950/states_simplified.topo.json
1,4M	../data/maps_conversion/1950/states.topo.json
1,7M	../data/maps_conversion/1956/states.json
172K	../data/maps_conversion/1956/states_simplified.topo.json
1,4M	../data/maps_conversion/1956/states.topo.json
1,7M	../data/maps_conversion/1962/states.json
144K	../data/maps_conversion/1962/states_simplified.topo.json
1,3M	../data/maps_conversion/1962/states.topo.json
1,7M	../data/maps_conversion/1970/states.json
184K	../data/maps_conversion/1970/states_simplified.topo.json
1,4M	../data/maps_conversion/1970/states.topo.json
1,8M	../data/maps_conversion/1980/states.json
184K	../data/maps_conversion/1980/states_simplified.topo.json
1,5M	../data/maps_conversion/1980/states.topo.json
1,8M	../data/maps_conversion/1991/states.json


In [6]:
%%bash
folder=../data/maps_conversion
dest=../data/deliver_json

mkdir $dest
for map_year in $(ls $folder); do
    echo "{\"year\": $map_year, \"map\":" \
        $(cat $folder/$map_year/*simplified.topo.json) \
        "}" > $dest/$map_year.json
done

In [7]:
%%bash
ls ../data/deliver_json/

1940.json
1950.json
1956.json
1962.json
1970.json
1980.json
1991.json


In [8]:
from glob import glob
import os
import json

In [9]:
contents = []
for file_path in glob('../data/deliver_json/*'):
    with open(file_path, 'r') as file:
        contents.append(json.load(file))

In [10]:
contents = sorted(contents, key=lambda d: d['year'])

In [11]:
with open('../data/deliver_json/all.json', 'w') as file:
    json.dump(contents, file, ensure_ascii=False)

In [12]:
ls ../data/deliver_json/

1940.json  1956.json  1970.json  1991.json
1950.json  1962.json  1980.json  all.json
